<a href="https://www.kaggle.com/code/gizemnalbantarslan/association-rule-based-recommender-system-armut?scriptVersionId=199068054" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

An online service platform brings service providers and service providers together.
It provides easy access to services such as cleaning, modification and transportation with a few touches on your computer or smart phone.
It is desired to create a product recommendation system with Association Rule Learning by using the data set containing the service users and the services and categories these users have received.

# DATASET

The data set consists of the services customers receive and the categories of these services.
It contains the date and time information of each service received.

* UserId: Customer number
* ServiceId: Anonymized services belonging to each category. (Example: Upholstery washing service under the cleaning category)
    A ServiceId can be found under different categories and refers to different services under different categories.
    (Example: Service with CategoryId 7 and ServiceId 4 is honeycomb cleaning, while service with CategoryId 2 and ServiceId 4 is furniture assembly)
* CategoryId: Anonymized categories. (Example: Cleaning, transportation, renovation category)
* CreateDate: The date the service was purchased

In [ ]:
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules
import datetime as dt
import seaborn as sns
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 500)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [ ]:
df = pd.read_csv("../input/armut-data/armut_data.csv")
df.columns = [col.lower() for col in df.columns]
df.head()

**DATA PREPARATION**

In [ ]:
df.info()
df['hizmet'] = [str(col[1]) + '_' + str(col[2]) for col in df.values]
df.head()

In [ ]:
df['createdate'] = pd.to_datetime(df['createdate'])
df['year'] = df['createdate'].dt.year
df['month'] = df['createdate'].dt.month
df['new_date'] = [str(col[-2]) + '_' + str(col[-1]) for col in df.values]
df['date'] = pd.to_datetime(df['createdate']).dt.to_period('M').astype('str')
df['sepet_id'] = [str(col[0]) + '_' + str(col[-1]) for col in df.values]

In [ ]:
df.head()

***Association Rules Are Produced***

In [ ]:
df_armut = df.groupby(['sepet_id', 'hizmet'])['serviceid'].count().unstack().fillna(0).applymap(lambda x: 1 if x !=0 else 0)

In [ ]:
frequent_items = apriori(df_armut, min_support=0.01, use_colnames=True)
frequent_items.sort_values('support', ascending=False)

rules = association_rules(frequent_items, metric='support', min_threshold=0.01)

In [ ]:
def arl_recommender(rules_df, product_id, rec_count=1):
    sorted_rules = rules_df.sort_values("lift", ascending=False)
    recommendation_list = []
    for i, product in enumerate(sorted_rules["antecedents"]):
        for j in list(product):
            if j == product_id:
                recommendation_list.append(list(sorted_rules.iloc[i]["consequents"])[0])

    return recommendation_list[0:rec_count]

arl_recommender(rules, '2_0', 1)
arl_recommender(rules, '2_0', 2)
arl_recommender(rules, '2_0', 3)